### SageMaker  Endpoint 部署ChatYuan
  
[ChatYuan](https://huggingface.co/ClueAI/ChatYuan-large-v1): 元语功能型对话大模型 , 这个模型可以用于问答、结合上下文做对话、做各种生成任务，包括创意性写作，也能回答一些像法律、新冠等领域问题。它基于PromptCLUE-large结合数亿条功能对话多轮对话数据进一步训练得到。

PromptCLUE-large在1000亿token中文语料上预训练，累计学习1.5万亿中文token，并且在数百种任务上进行Prompt任务式训练。针对理解类任务，如分类、情感分析、抽取等，可以自定义标签体系；针对多种生成任务，可以进行采样自由生成。

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [ ]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

In [ ]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

In [4]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatyuan')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatyuan')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-1-596030579944/chatyuan/assets/model.tar.gz


### 设置模型推理参数
1. model_name: Huggingface diffusers models (not support single check point format)
2. model_args: diffuser StableDiffusionPipeline init arguments

In [5]:
model_name = None
entry_point = 'inference.py'
framework_version = '1.12'
py_version = 'py38'
bucket='m.flowq.io'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

In [6]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = 'inference.py',
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

### 部署 SageMaker Endpoint
部署推理服务

In [7]:
endpoint_name = None
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

------------!

### ChatYuan 测试


In [ ]:
inputs= {
    "ask": "帮我写一个请假条，我因为新冠不舒服，需要请假3天，请领导批准"
    #"ask": "帮我写一个请假条，我因为新冠不舒服，需要请假3天，请领导批准"
    #"ask": "学前教育专业岗位实习中，在学生方面会存在问题，请提出改进措施。800字"
}

response = predictor.predict(inputs)
print(response["answer"])


### 删除SageMaker  Endpoint
删除推理服务

In [9]:
predictor.delete_endpoint()